In [5]:
from relapse_prediction import constants

In [6]:
from sklearn.metrics import auc
from tqdm.auto import tqdm
import pandas as pd
import pickle

# 1. Cercare features : 

In [9]:
df_cercare_auc = pd.DataFrame(columns=["patient", "label", "imaging", "feature", "AUC"])

for patient in tqdm(constants.list_patients):
    for label in ["L3R", "L3R_5x5x5", "L3R - (L1 + L3)", "L3R - (L1 + L3)_5x5x5"]:
        for imaging in constants.L_CERCARE_MAPS:
            for feature in [None, "mean_5x5x5"]:
                
                
                feature_col = f"{imaging}_{feature}" if feature is not None else imaging
                with open(constants.dir_thresholds / patient / label / f"{feature_col}_quantized.pickle", "rb") as f:
                    dict_thresholds = pickle.load(f)
                
                fpr, tpr = dict_thresholds["fpr"], dict_thresholds["tpr"]
                auc_value = auc(fpr, tpr)
                
                df_cercare_auc.loc[len(df_cercare_auc)] = [patient, label, imaging, feature, auc_value]

df_cercare_auc.to_csv("cercare_auc_values_per_patient.csv", index=False)

  0%|          | 0/104 [00:00<?, ?it/s]

In [27]:
df_cercare_auc["feature"].fillna("", inplace=True)
df_cercare_auc.groupby(["label", "imaging", "feature"])["AUC"].mean().reset_index()

,label,imaging,feature,AUC
0,L3R,COV,,0.566224
1,L3R,COV,mean_5x5x5,0.575033
2,L3R,CTH,,0.681553
3,L3R,CTH,mean_5x5x5,0.696827
4,L3R,Delay,,0.608217
5,L3R,Delay,mean_5x5x5,0.623822
6,L3R,OEF,,0.635342
7,L3R,OEF,mean_5x5x5,0.647468
8,L3R,rCBV,,0.518676
9,L3R,rCBV,mean_5x5x5,0.520338


# 2. MRI features

In [12]:
df_mri_auc = pd.DataFrame(columns=["patient", "label", "imaging", "feature", "norm", "AUC"])

for patient in tqdm(constants.list_patients):
    for label in ["L3R", "L3R_5x5x5", "L3R - (L1 + L3)", "L3R - (L1 + L3)_5x5x5"]:
        for imaging in constants.L_IRM_MAPS:
            for feature in [None, "mean_5x5x5"]:
                for norm in ["z_score", "min_max", "max"]:
                
                    feature_col = f"{imaging}_{feature}" if feature is not None else imaging
                    with open(constants.dir_thresholds / patient / label / f"{feature_col}_{norm}_normalized.pickle", "rb") as f:
                        dict_thresholds = pickle.load(f)
                    
                    fpr, tpr = dict_thresholds["fpr"], dict_thresholds["tpr"]
                    auc_value = auc(fpr, tpr)
                    
                    df_mri_auc.loc[len(df_mri_auc)] = [patient, label, imaging, feature, norm, auc_value]

df_mri_auc.to_csv("mri_auc_values_per_patient.csv", index=False)

  0%|          | 0/104 [00:00<?, ?it/s]

In [28]:
df_mri_auc["feature"].fillna("", inplace=True)
df_mri_auc.groupby(["label", "imaging", "feature", "norm"])["AUC"].mean().reset_index()

,label,imaging,feature,norm,AUC
0,L3R,FLAIR,,max,0.657369
1,L3R,FLAIR,,min_max,0.657336
2,L3R,FLAIR,,z_score,0.657522
3,L3R,FLAIR,mean_5x5x5,max,0.702652
4,L3R,FLAIR,mean_5x5x5,min_max,0.702659
...,...,...,...,...,...
67,L3R_5x5x5,T1CE,,min_max,0.602975
68,L3R_5x5x5,T1CE,,z_score,0.603129
69,L3R_5x5x5,T1CE,mean_5x5x5,max,0.673776
70,L3R_5x5x5,T1CE,mean_5x5x5,min_max,0.673736
